# Laboratory work #1
For beginning importing libraries.

In [9]:
# Importing scikit-learn libraries
from sklearn.metrics import classification_report, confusion_matrix

# Importing other libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import zipfile

#### Downloading Dataset
Our dataset is Possible Asteroid Impacts with Earth (from [kaggle](https://www.kaggle.com/datasets/nasa/asteroid-impacts))

## Preprocessing Dataset

In [14]:
# Loading dataset to pandas DataFrame
asteroid_orbits_df = pd.read_csv("datasets\\asteroid_dataset\\orbits.csv")
asteroid_impacts_df = pd.read_csv("datasets\\asteroid_dataset\\impacts.csv")

In [16]:
# Getting data from dataframe
print(asteroid_orbits_df.head())
print(asteroid_impacts_df.head())
# Getting names of column
print("==>Column names<==")
print(f"=>orbits names: {asteroid_orbits_df.columns.tolist()}")
print(f"=>impact names: {asteroid_impacts_df.columns.tolist()}")
# Getting size of dataset
print("==>Dataset size<==")
print(f"orbits shape={asteroid_orbits_df.shape}")
print(f"impact shape={asteroid_impacts_df.shape}")

    Object Name Object Classification  Epoch (TDB)  Orbit Axis (AU)  \
0      433 Eros         Amor Asteroid        57800           1.4579   
1    719 Albert         Amor Asteroid        57800           2.6385   
2    887 Alinda         Amor Asteroid        57800           2.4787   
3  1036 Ganymed         Amor Asteroid        57800           2.6628   
4     1221 Amor         Amor Asteroid        57800           1.9191   

   Orbit Eccentricity  Orbit Inclination (deg)  Perihelion Argument (deg)  \
0              0.2226                  10.8277                   178.8050   
1              0.5479                  11.5822                   156.1409   
2              0.5671                   9.3561                   350.3482   
3              0.5338                  26.6929                   132.4690   
4              0.4356                  11.8795                    26.6572   

   Node Longitude (deg)  Mean Anomoly (deg)  Perihelion Distance (AU)  \
0              304.3265            31

In [20]:
# Renaming some columns
asteroid_orbits_df = asteroid_orbits_df.rename(columns={"Object Name": "Name"})
asteroid_impacts_df = asteroid_impacts_df.rename(columns={"Object Name": "Name"})

In [22]:
# Choosing some columns for analysis
asteroid_orbits_df = asteroid_orbits_df.loc[:, ["Name", "Orbit Axis (AU)", "Orbit Eccentricity",
                                                "Orbit Inclination (deg)", "Asteroid Magnitude"]]
asteroid_impacts_df = asteroid_impacts_df.loc[:, ["Name", "Possible Impacts", "Cumulative Impact Probability",
                                                  "Asteroid Velocity", "Asteroid Diameter (km)",
                                                  "Asteroid Magnitude", "Maximum Palermo Scale",
                                                  "Maximum Torino Scale"]]

### Combining DataFrames

In [73]:
asteroid_df = pd.concat([asteroid_orbits_df, asteroid_impacts_df])
print(asteroid_df.loc[asteroid_df["Name"].isin(["2008 US", "2008 XK", "6239 Minos"]), ["Name", "Possible Impacts"]])

       Name  Possible Impacts
2   2008 US              30.0
12  2008 XK             171.0


### NaN Checking & Preprocessing

In [74]:
# Checking NaN variables in our dataset
print("Have NaN variables" if asteroid_df.isnull().values.any() else
      "Don't have NaN variables")
# Counting NaN variables
print(f"Count of NaN: {asteroid_df.isnull().sum().sum()}")
# Checking NaN per columns
print("==>Count of Nan per column<==")
print(asteroid_df.isnull().sum())

Have NaN variables
Count of NaN: 95860
==>Count of Nan per column<==
Name                                 0
Orbit Axis (AU)                    683
Orbit Eccentricity                 683
Orbit Inclination (deg)            683
Asteroid Magnitude                   1
Possible Impacts                 15635
Cumulative Impact Probability    15635
Asteroid Velocity                15635
Asteroid Diameter (km)           15635
Maximum Palermo Scale            15635
Maximum Torino Scale             15635
dtype: int64


In [89]:
# Preprocessing data gaps (NaN variables)
asteroid_orbits_df = asteroid_orbits_df.loc[~asteroid_orbits_df["firstname"].isna()
                                              & ~asteroid_orbits_df["surname"].isna()]
print(asteroid_orbits_df.head())

      id               firstname    surname category
896  524           Martin Luther   King Jr.    peace
903  463                Fr_d_ric      Passy    peace
904  466          William Randal     Cremer    peace
905  462              Jean Henry     Dunant    peace
906  475  Auguste Marie Franois  Beernaert    peace


## Visualising Dataset

In [70]:
# Visualising with heatmap
plt.style.use("seaborn")
plt.figure(figsize=(10, 10))
heat_map = sns.heatmap(nobel_prize_winners, linewidths=1, annot=True)
plt.title("HeatMap")
plt.show()

C:\Users\Who\AppData\Local\Temp\ipykernel_4348\2893843219.py:2: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use("seaborn")


ValueError: zero-size array to reduction operation fmin which has no identity

<Figure size 1000x1000 with 0 Axes>